1) instale a biblioteca eodag para consultar as cenas que estão em sua área de estudo (Obs: só é necessário instalar uma vez, depois comente a linha)

In [1]:
!pip install eodag

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.0 MB/s eta 0:00:004.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 249.6 kB/s eta 0:00:00m eta 0:00:010:02:09
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 848.1 kB/s eta 0:00:00 kB/s eta 0:00:01:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 750.0 kB/s eta 0:00:000:00:010:00:01:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 710.7 kB/s eta 0:00:00 kB/s eta 0:00:01:04
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.69
    Uninstalling botocore-1.34.69:
      Successfully uninstalled botocore-1.34.69
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.37.20 which is incompatible.


2. Importe as bibliotecas e defina os limites máximos e mínimos (lat, lon) da sua área de estudo

In [5]:
#Importando bibliotecas necessárias
import os
from eodag import EODataAccessGateway, setup_logging
import datetime
from dateutil import parser
import time
import requests
from tqdm import tqdm
import sys

#Acessando dados
dag = EODataAccessGateway()

#Criando coordenadas da minha área de estudo
coords = [-43.29238462775601, -23.01198041390091, -43.01635313361538, -22.68040512596414] #Insera nesta ordem: lonmin, latmin, lonmax, latmax

3. Looping para procurar as cenas que estão dentra da geometria para um range de data específico

In [12]:
start_date = '2023-06-26'
end_date = '2025-03-01'
search_results = dag.search(
    productType='S3_EFR',
    geom={'lonmin': coords[0], 'latmin': coords[1], 'lonmax': coords[2], 'latmax': coords[3]},
    start=start_date,
    end=end_date,
    #provider='creodias',
)

Error while searching on provider creodias (ignored):
Traceback (most recent call last):
  File "/home/de-sousa/anaconda3/lib/python3.12/site-packages/eodag/api/core.py", line 1908, in _do_search
    download_plugin = self._plugins_manager.get_download_plugin(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/de-sousa/anaconda3/lib/python3.12/site-packages/eodag/plugins/manager.py", line 242, in get_download_plugin
    self._build_plugin(product.provider, download, Download),
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/de-sousa/anaconda3/lib/python3.12/site-packages/eodag/plugins/manager.py", line 466, in _build_plugin
    plugin_class = EODAGPluginMount.get_plugin_by_class_name(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/de-sousa/anaconda3/lib/python3.12/site-packages/eodag/plugins/base.py", line 55, in get_plugin_by_class_name
    raise PluginNotFoundError(
eodag.utils.exceptions.PluginNotFou

In [13]:
search_results

SearchResult (0)


In [9]:
# Parâmetros iniciais
start_date = '2023-06-26'
end_date = '2025-03-01'
all_results = []

while parser.parse(start_date) < parser.parse(end_date):
    # Fazer a busca
    search_results = dag.search(
        productType='S3_EFR',
        geom={'lonmin': coords[0], 'latmin': coords[1], 'lonmax': coords[2], 'latmax': coords[3]},
        start=start_date,
        end=end_date,
        provider='creodias',
    )

    if not search_results:
        print("Nenhum novo resultado encontrado. Encerrando...")
        break

    # Armazena os resultados encontrados
    all_results.extend(search_results)

    # Obtém a última data retornada
    last_entry = search_results[-1]
    last_date_str = str(last_entry).split("id=")[1][16:24]  # Ajuste conforme necessário

    # Exibir para conferir o formato
    print(f"Data extraída: {last_date_str}")

    try:
        # Tentar converter a data
        last_date = datetime.datetime.strptime(last_date_str, '%y%m%d')  # Formato esperado: ano (2 dígitos), mês e dia
    except ValueError:
        # Ajuste caso o formato seja diferente (por exemplo, com ano completo)
        last_date = datetime.datetime.strptime(last_date_str, '%Y%m%d')  # Ajuste para o formato completo do ano

    # Define o próximo início um dia após a última data
    start_date = (last_date + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

    print(f"Nova busca a partir de {start_date}")

print(f"Total de produtos encontrados: {len(all_results)}")

Error while searching on provider creodias (ignored):
Traceback (most recent call last):
  File "/home/de-sousa/anaconda3/lib/python3.12/site-packages/eodag/api/core.py", line 1908, in _do_search
    download_plugin = self._plugins_manager.get_download_plugin(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/de-sousa/anaconda3/lib/python3.12/site-packages/eodag/plugins/manager.py", line 242, in get_download_plugin
    self._build_plugin(product.provider, download, Download),
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/de-sousa/anaconda3/lib/python3.12/site-packages/eodag/plugins/manager.py", line 466, in _build_plugin
    plugin_class = EODAGPluginMount.get_plugin_by_class_name(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/de-sousa/anaconda3/lib/python3.12/site-packages/eodag/plugins/base.py", line 55, in get_plugin_by_class_name
    raise PluginNotFoundError(
eodag.utils.exceptions.PluginNotFou

Nenhum novo resultado encontrado. Encerrando...
Total de produtos encontrados: 0


4. Realizando download das imagens encontradas. Obs: defina o diretório para onde você deseja descarregar as imagens. ATTENZIONNE: É NECESSÁRIO CRIAR UM LOGIN E SENHA EM: https://browser.dataspace.copernicus.eu/

In [4]:
#Escreva abaixo o seu login-senha do copernicus browser
user = 'username'
password = 'password'

#Abaixo defina o diretório em que você deseja descarregar as imagens (por padrão vou deixar a própria pasta onde está o código, mas isso não é recomendável
download_directory = '.'

#Código de download
def get_access_token(username, password):
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Access token creation failed. Response from the server was: {r.json()}"
        )
    return r.json()["access_token"]

def download_copernicus_product(access_token, name_file, output_directory=".", output_name=None):
    from tqdm import tqdm
    extent_file = {'S2': 'SAFE', 'S3': 'SEN3'}

    json_data = requests.get(
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Name eq '{name_file}'"
    ).json()

    id_product = pd.DataFrame.from_dict(json_data["value"]).head(1)["Id"][0]

    url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({id_product})/$value"

    headers = {"Authorization": f"Bearer {access_token}"}

    session = requests.Session()
    session.headers.update(headers)
    response = session.get(url, headers=headers, stream=True)

    file_size = int(response.headers.get("content-length", 0))
    block_size = 1024

    output_name = output_name or f"{name_file}.zip"
    output_path = os.path.join(output_directory, output_name)

    with open(output_path, "wb") as file, tqdm(
        desc="Downloading",
        total=file_size,
        unit="B",
        unit_scale=True,
        unit_divisor=1024,
        bar_format="{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} {unit}",
        dynamic_ncols=True,
        colour="BLUE",
        leave=False,
    ) as bar:
        for chunk in response.iter_content(chunk_size=block_size):
            if chunk:
                file.write(chunk)
                bar.update(len(chunk))

path_list_zip = os.listdir(download_directory)
files_zip = [arquivo[:-4] for arquivo in path_list_zip if arquivo.endswith('.zip')]

download_count = 0

for name_file in all_results:
    name_file = name_file+'.SEN3'

    if name_file in files_zip:
        print(f"{name_file} já foi baixado anteriormente")
        continue  # pula os produtos já baixados

    if download_count == len(result_df):
        # Reinicia a sessão do Google Colab após 4 downloads
        print(f"Encerrando sessão após baixar {len(all_results)} produtos...")
        #download_count = 0
        break

    try:
        access_token = get_access_token(user, password)
        print(name_file)
        download_copernicus_product(access_token, name_file, download_directory)
        download_count += 1

    except Exception as e:
        print(f"{name_file} falhou no download: {e}")
        # Exclui o arquivo que estava sendo baixado em caso de erro
        file_path = f"./l1_scenes/{name_file}.zip"
        if os.path.exists(file_path):
            try:
                os.remove(file_path)
                print(f"Arquivo {file_path} excluído devido a erro no download.")
            except Exception as delete_error:
                print(f"Falha ao tentar excluir {file_path}: {delete_error}")

FileNotFoundError: [Errno 2] No such file or directory: 'list_scene_s3.csv'